### Distance Contraint

We try to find a correction $\Delta \mathbf{p}$ so that the $\textbf{Contraint Function: }$
$$
\begin{align*}
C_d \left(\mathbf{p} + \Delta \mathbf{p} \right) &= \mathbf{0} \\
C_d \left(\mathbf{p} + \Delta \mathbf{p} \right) &\approx C_d (\mathbf{p}) + \nabla_{\mathbf{p}}C_d(\mathbf{p}) \cdot \Delta \mathbf{p} = \mathbf{0}
\end{align*}
$$
$$$$
Restricting $\Delta \mathbf{p}$ to be in the direction of $\nabla_{\mathbf{p}}$ means choosing a $\lambda$ so that
$$\Delta \mathbf{p} = \lambda \nabla_{\mathbf{p}}C_d(\mathbf{p})$$

In [14]:
%%HTML
<span><img src="distance_constraint1.png" width="500px"/></span>

The correction for an individual point $i$ is given by:
$$ \Delta \mathbf{p}_i = -s \nabla_{\mathbf{p}_i}C_d(\mathbf{p}_1,\dots,\mathbf{p}_n) $$
where the scaling factor $s$
$$ s = \frac{C_d(\mathbf{p}_1,\dots,\mathbf{p}_n)}{\sum |\nabla_{\mathbf{p}_j}C_d(\mathbf{p}_1,\dots,\mathbf{p}_n)|^2}$$
is the same for all points. In case that the points have different masses, the term is weighted by the inverse masses $w_i = \frac{1}{m_i}$
$$ s = \frac{C_d(\mathbf{p}_1,\dots,\mathbf{p}_n)}{\sum {w_j} |\nabla_{\mathbf{p}_j}C_d(\mathbf{p}_1,\dots,\mathbf{p}_n)|^2}$$
which leads to the weighted correction:
$$ \Delta \mathbf{p}_i = -s w_i \nabla_{\mathbf{p}_i}C_d(\mathbf{p}_1,\dots,\mathbf{p}_n) $$

### Example Distance Constraint
For the distance constraint $C_d(\mathbf{p}_1,\mathbf{p}_2) = |\mathbf{p}_1 -\mathbf{p}_2| - d$ we have the derivatives:
$$
\begin{align}
\nabla_{\mathbf{p}_1} C(\mathbf{p}_1,\mathbf{p}_2)&= \mathbf{n} \\
\nabla_{\mathbf{p}_2} C(\mathbf{p}_1,\mathbf{p}_2)&= -\mathbf{n}
\end{align}
$$
where 
$$
\mathbf{n} = \frac{\mathbf{p}_1 -\mathbf{p}_2}{|\mathbf{p}_1 -\mathbf{p}_2|}.
$$
The scaling factor $s$ here is
$$
\frac{|\mathbf{p}_1 -\mathbf{p}_2| - d}{w_1 + w_2}
$$
because $|\nabla_{\mathbf{p}_j}C_d(\mathbf{p}_1,\dots,\mathbf{p}_n)|^2=|\mathbf{n}|=1$ 

So the final corrections are:
$$
\begin{align}
\Delta \mathbf{p}_1 = -\frac{w_1 |\mathbf{p}_1 -\mathbf{p}_2| - d}{w_1 + w_2} \frac{\mathbf{p}_1 -\mathbf{p}_2}{|\mathbf{p}_1 -\mathbf{p}_2|}\\
\Delta \mathbf{p}_2 = +\frac{w_2 |\mathbf{p}_1 -\mathbf{p}_2| - d}{w_1 + w_2} \frac{\mathbf{p}_1 -\mathbf{p}_2}{|\mathbf{p}_1 -\mathbf{p}_2|}\\
\end{align}
$$

In [13]:
%%HTML
<span><img src="bending_constraint.svg" width="500px"/></span>

### Bending Contraint




In [40]:
%%latex
$$\mathbf{n}_1 = \left(\mathbf{p}_2-\mathbf{p}_1\right) \times \left(\mathbf{p}_3-\mathbf{p}_1\right)$$
$$\mathbf{n}_2 = \left(\mathbf{p}_2-\mathbf{p}_1\right) \times \left(\mathbf{p}_4-\mathbf{p}_1\right)$$
$$ $$
$$\textbf{Contraint Function}$$
$$c_b = \left(\mathbf{p}_1,\mathbf{p}_2,\mathbf{p}_3, \mathbf{p}_4\right) = \arccos \left( \frac{\mathbf{n}_1}{|\mathbf{n}_1|} \cdot \frac{\mathbf{n}_2}{|\mathbf{n}_2|} \right) - \theta_0$$

<IPython.core.display.Latex object>

### The Dihedral Angle and the Bending Constraint
The dihedral angle $\theta$ is the angle between to adjacent triangles. The angle $\theta_0$ is the angle in the 'resting' position of a deformable mesh. So the bending constraint $c_b$ measures the difference between the resting angle $\theta_0$ and
the current dihedral angle $\theta$.

In [41]:
%%HTML
<p><img src="bending_constraint2.svg" width="500px"/></p>

## Position Based Dynamics Algorithm

```c++
void dampVelocities(vector &v);
void solveConstraints(vector &constraints, vector &vertices);

std::vector<VertexType> tempPos;
std::vector<ConstraintType> constraints;

while(simulationRunning) {   
  for(auto &v : vertices)
    // temporary velocity update  
    v.vel = v.vel + dt * v.weight * v.extForce;  
  dampVelocities(vertices);
    
  for(int i(0); i < vertices.size(); ++i)     
    // temporary position update  
    tempPos[i] = vertices[i].pos + dt * vertices[i].vel;    
  
  // Solve the constraints to get the corrections
  for(int i(0); i < solverIterations; ++i)         
    solveConstraints(constraints, vertices);    
    
  // correct the temporary positions and velocities  
  for(int i(0); i < vertices.size(); ++i)
    VertexType &v = vertices[i];  
    v.vel = (tempPos[i] - v.pos)/dt;
    v.pos = tempPos[i];
        
}
```